In [39]:
from langchain.llms.openai import OpenAI
from langchain.memory import  ConversationBufferMemory
from langchain.memory import  ConversationBufferWindowMemory
from langchain.memory import ConversationSummaryMemory
from langchain.memory import ConversationSummaryBufferMemory
from langchain.callbacks import StreamingStdOutCallbackHandler
from dotenv import load_dotenv
import os


In [40]:
load_dotenv()
SERVICE_KEY = os.getenv('OPEN_API_KEY')

# ConversationBufferMemory 는 대화에대한 것을 모두 memory에 넣어줘야해서 비용이 커지는 메모리임 잘사용안함.
# ConversationBufferWindowMemory 는 모든 대화를 메모리에 가지고 있지않음.

llm = OpenAI(openai_api_key=SERVICE_KEY, model_name="gpt-3.5-turbo-1106", temperature=0.1, streaming=True, callbacks=[StreamingStdOutCallbackHandler()])

memory = ConversationBufferMemory(return_message=True)
memory.save_context({"input" : "HI!"}, {"output": "How are you ?"})
memory.load_memory_variables({})


C:\Users\smkim\AppData\Local\anaconda3\envs\STT\Lib\site-packages\langchain\llms\openai.py:216: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
C:\Users\smkim\AppData\Local\anaconda3\envs\STT\Lib\site-packages\langchain\llms\openai.py:811: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


{'history': 'Human: HI!\nAI: How are you ?'}

In [41]:
memory.save_context({"input" : "HI!"}, {"output": "How are you ?"})
memory.load_memory_variables({})


{'history': 'Human: HI!\nAI: How are you ?\nHuman: HI!\nAI: How are you ?'}

In [42]:
memory.save_context({"input" : "HI!"}, {"output": "How are you ?"})
memory.load_memory_variables({})


{'history': 'Human: HI!\nAI: How are you ?\nHuman: HI!\nAI: How are you ?\nHuman: HI!\nAI: How are you ?'}

In [43]:
## ConversationBufferWindowMemory 는 최근 k 갯수만큼 메모리 할수 있음. 4개까지 킵하도록 설정 
memory2 = ConversationBufferWindowMemory(return_message=True, k=4)
def add_message(memory,  input, output):
    memory.save_context({"input" : input}, {"output": output})
    
add_message(memory2, 1,1)
add_message(memory2, 2,2)
add_message(memory2, 3,3)
add_message(memory2, 4,4)
add_message(memory2, 5,5)

memory2.load_memory_variables({})

{'history': 'Human: 2\nAI: 2\nHuman: 3\nAI: 3\nHuman: 4\nAI: 4\nHuman: 5\nAI: 5'}

In [44]:
# ConversationSummaryMemory은 llm 을 활용하여 Memory에 요약해서 셋업할수있음.
memory3 = ConversationSummaryMemory(llm=llm)
add_message(memory3, "hi im sungmin, i live in south korea", 'Wow that is cool!')
add_message(memory3, "South Korea is so pretty", 'I wish i could go!!!')


memory3.load_memory_variables({})

{'history': "The human introduces themselves as Sungmin from South Korea. The AI responds with enthusiasm, finding Sungmin's location interesting and expressing a wish to visit."}

In [45]:

## ConversationSummaryBufferMemory은 max token 에 도달했을때까지 메세지를 memory에 담아두고 요약해서 리턴한다.
memory4 = ConversationSummaryBufferMemory(
    llm = llm, max_token_limit=100, return_message=True
)

add_message(memory4, "hi im sungmin, i live in south korea", 'Wow that is cool!')

memory4.load_memory_variables({})

add_message(memory4, "South Korea is so pretty", 'I wish i could go!!!')

memory4.load_memory_variables({})

{'history': 'Human: hi im sungmin, i live in south korea\nAI: Wow that is cool!\nHuman: South Korea is so pretty\nAI: I wish i could go!!!'}

In [46]:
add_message(memory4, "How far is Korea from Argentina?", "I don't know! Super far!")

memory4.load_memory_variables({})

add_message(memory4, "How far is Brazil from Argentina", "I don't know! Super far!")

memory4.load_memory_variables({})

{'history': "Human: hi im sungmin, i live in south korea\nAI: Wow that is cool!\nHuman: South Korea is so pretty\nAI: I wish i could go!!!\nHuman: How far is Korea from Argentina?\nAI: I don't know! Super far!\nHuman: How far is Brazil from Argentina\nAI: I don't know! Super far!"}

In [48]:
add_message(memory4, "How far is USA from Argentina", "I don't know! Super far!")

memory4.load_memory_variables({})

add_message(memory4, "How far is USA from Korea", "I don't know! Super far!")

memory4.load_memory_variables({})

The human introduces themselves as Sungmin from South Korea.The human introduces themselves as Sungmin from South Korea, and the AI expresses admiration for that. Sungmin then comments on the beauty of South Korea.

{'history': "System: The human introduces themselves as Sungmin from South Korea, and the AI expresses admiration for that. Sungmin then comments on the beauty of South Korea.\nAI: I wish i could go!!!\nHuman: How far is Korea from Argentina?\nAI: I don't know! Super far!\nHuman: How far is Brazil from Argentina\nAI: I don't know! Super far!\nHuman: How far is USA from Argentina\nAI: I don't know! Super far!\nHuman: How far is USA from Argentina\nAI: I don't know! Super far!\nHuman: How far is USA from Korea\nAI: I don't know! Super far!"}